In [ ]:
import pandas as pd
import numpy as np

In [ ]:
"""# old labels
#path = ""
path ="old_labels/"
test = pd.read_csv(path + "test_subset.csv")
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_3 = pd.read_csv(path + "train_third_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_1["label"] = pd.to_numeric(train_1["label"], errors='coerce')
train = pd.concat([train_1,train_2,train_3],axis=0)
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

test = test.reset_index()
train=train.reset_index()
val = val.reset_index()

train.head()

In [5]:
# new labels
#path = "new_labels/"
path = ""
test = pd.read_csv(path + "test_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
test["label"] = pd.to_numeric(test["label"], errors='coerce')
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
extra_neg = pd.read_csv(path + "extra_neg.csv")[["Unnamed: 0",'restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect1','new_aspect2']]
extra_neg = extra_neg.rename(columns= {'new_aspect1':'new_aspect_1', 'new_aspect2':'new_aspect_2'})
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

index = extra_neg.loc[extra_neg["Unnamed: 0"] == 0].index

test = pd.concat([test,extra_neg.iloc[:index[1],1:]],axis=0)
train = pd.concat([train_1,train_2,extra_neg.iloc[index[1] + 1:index[2],1:]],axis=0)
val = pd.concat([val,extra_neg.iloc[index[2] + 1:,1:]],axis=0)
train.shape,val.shape,test.shape

((8348, 10), (3055, 10), (3040, 10))

In [ ]:
train.phrase[500:590]

In [ ]:
for data in [train_1,train_2,train_3,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

In [ ]:
test.head()

####  Data Preprocessing

1. Clean Phrase (Utils) 
2. Filter the phrase column where there is an empty string 
3. Combine the aspects into 1 & aggregate the sentiment scores by averaging then sign(>0 = 1,=0 =0 <0 = -1)


In [ ]:
# clean phrases
train["phrase_lemma"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))
val["phrase_lemma"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))
test["phrase_lemma"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))

train["phrase_stem"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))
val["phrase_stem"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))
test["phrase_stem"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))

train["phrase_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))
val["phrase_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))
test["phrase_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))

train["phrase_lemma_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))
val["phrase_lemma_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))
test["phrase_lemma_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))

train["phrase_stem_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))
val["phrase_stem_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))
test["phrase_stem_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))

train["phrase_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))
val["phrase_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))
test["phrase_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))

train["phrase_lemma_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))
val["phrase_lemma_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))
test["phrase_lemma_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))

train["phrase_stem_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))
val["phrase_stem_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))
test["phrase_stem_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))

train.phrase = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))
val.phrase = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))
test.phrase = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))

In [ ]:
# filter out rows where there are no characters
train = train.loc[(train.phrase.str.len() > 0)]
val = val.loc[(val.phrase.str.len() > 0)]
test = test.loc[(test.phrase.str.len() > 0)]

In [ ]:
# if nan label, replace with 0
train.label = train.label.apply(lambda x: 0 if np.isnan(x) else x)
val.label = val.label.apply(lambda x: 0 if np.isnan(x) else x)
test.label = test.label.apply(lambda x: 0 if np.isnan(x) else x)

In [ ]:
trainagg = train.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma":" ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                
                                                                                                "label": np.mean})                                                                                                
trainagg = trainagg.reset_index()
trainagg.label = trainagg.label.map(np.sign)

In [ ]:
valagg = val.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                               "phrase_lemma":" ".join,
                                                                                               "phrase_stem":" ".join,
                                                                                               "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                  
                                                                                               "label": np.mean})   
valagg = valagg.reset_index()
valagg.label = valagg.label.map(np.sign)

In [ ]:
testagg = test.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                               "phrase_lemma":" ".join,
                                                                                               "phrase_stem":" ".join,
                                                                                               "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                 
                                                                                               "label": np.mean})   
testagg = testagg.reset_index()
testagg.label = testagg.label.map(np.sign)
testagg.label.value_counts()

In [ ]:
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [ ]:
path = ""
trainagg.to_csv(path + "train_newpreproc_emoticon.csv")
valagg.to_csv(path + "val_newpreproc_emoticon.csv")
testagg.to_csv(path + "test_newpreproc_emoticon.csv")

###### Text Vectorisation --> used on data2
1. BoW
2. Tf-idf
3. Fasttext - can handle oov words https://www.quora.com/How-does-fastText-output-a-vector-for-a-word-that-is-not-in-the-pre-trained-model (not used)


In [ ]:
trainagg = pd.read_csv("train_newpreproc_emoticon.csv")
valagg = pd.read_csv("val_newpreproc_emoticon.csv")
testagg = pd.read_csv("test_newpreproc_emoticon.csv")
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [ ]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
bow_train = bow.fit_transform(trainvalagg.phrase)
bow_test = bow.transform(testagg.phrase)

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
tfidf_train = tfidf.fit_transform(trainvalagg.phrase)
tfidf_test = tfidf.transform(testagg.phrase)

In [ ]:
"""
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

# Preprocess data for fasttext word embeddings training
train_clean = [process_text(sentence) for sentence in tqdm(trainvalagg.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
train_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(train_clean)]

# Preprocess data for fasttext word embeddings testing data
test_clean = [process_text(sentence) for sentence in tqdm(test2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
test_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(test_clean)]

# Train Fasttext word embeddings on training data
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 

fast_Text_model = FastText(train_word_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)
"""

In [ ]:
"""
# Save fastText gensim model
fast_Text_model.save("ftmodel/ft_model_train")
# Load saved gensim fastText model
fast_Text_model = Word2Vec.load("ftmodel/ft_model_train") """

In [ ]:
"""
# convert sentence --> average(each word embedding)
fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
# convert sentence --> average(each word embedding)
fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]
"""

In [ ]:
"""
fasttext_train_processed = []
for row in fasttext_train:
    try:
        fasttext_train_processed.append(list(row))
    except:
        fasttext_train_processed.append([0]*300)

fasttext_test_processed = []
for row in fasttext_test:
    try:
        fasttext_test_processed.append(list(row))
    except:
        fasttext_test_processed.append([0]*300)
"""

In [ ]:
"""
fasttext_train_processed = np.array(fasttext_train_processed).reshape(-1,300)
fasttext_test_processed = np.array(fasttext_test_processed).reshape(-1,300)
"""

##### Modelling

1. Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression

(Baseline) 1. Naive Bayes -Support Vector Machines (NBSVM)- Logistic Regression

In [ ]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train]
all_test = [bow_test,tfidf_test]
all_vectorisers = ["BoW","TfIdf"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,trainvalagg.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, logreg.predict(test),digits=4)) 
    
    svm.fit(train,trainvalagg.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, svm.predict(test),digits=4)) 

    # Fasttest dont work w NaiveBayes
    nb.fit(train,trainvalagg.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, nb.predict(test),digits=4))    

In [ ]:
(34/383)*0.2769  + (104/383)*0.5693 + (245/383)*0.6531

In [ ]:
(104/775)*0.0917 + (218/775)*0.2963 + (453/775)*0.7483